In [2]:
import duckdb
import yaml

In [2]:
def get_icd_codes(medpar_hospitalizations_prefix, start_year, end_year, output_file):
    """
    Extracts distinct ICD-10 codes starting with 'J' across multiple years.
    Saves them to a YAML File with proper formatting. 

    Params:
    medpar_hospitalizations_prefix (str): File prefix for the Parquet files.
    start_year (int): Start year (e.g., 2015).
    end_year (int): End year (e.g., 2018).
    output_file (str): Path to save the output YAML file.
    """

    conn = duckdb.connect()
    icd10_codes = []

    for year in range(start_year, end_year+1):
        file = f"{medpar_hospitalizations_prefix}_{year}.parquet"
        print(f"Processing {file}...")

        #sql query
        icd_code_query = f"""
        WITH icd_diag AS (
        SELECT UNNEST(diagnoses) as diag
        FROM '{file}'
        )

        SELECT DISTINCT diag 
        FROM icd_diag
        WHERE diag LIKE 'J%'
        """

        #Execute the query and fetch results 
        icd_codes_df = conn.execute(icd_code_query).fetchdf()

        icd10_codes.extend(icd_codes_df['diag'].tolist())

    #close the connection 
    conn.close()

    # Remove duplicates from the ICD-10 codes list, if any, and join them as a comma-separated string
    icd10_codes = '[' + ', '.join(f'"{code}"' for code in sorted(set(icd10_codes))) + ']'

    # Construct the final dictionary with the icd10 codes as a comma-separated string
    icd_codes_dict = {
        'respiratory_diseases': {
            'long_name': 'Diseases of the Respiratory System',
            'icd10': icd10_codes
        }
    }

    # Convert the dictionary to a YAML string
    yaml_str = yaml.dump(icd_codes_dict, default_flow_style=False, sort_keys=False)
    yaml_str = yaml_str.replace("'", "")  # Remove single quotes

    # Save the YAML File 
    with open(output_file, 'w') as yaml_file:
        yaml_file.write(yaml_str)


In [ ]:
medpar_hospitalizations_prefix = '../data/input/mbsf_medpar_denom/medpar_hospitalizations'
output_file = '../conf/icd_codes/icd_codes_8.yml'
get_icd_codes(medpar_hospitalizations_prefix, 2015, 2018, output_file)

In [ ]:
## alternative query approach

# ICD-10 codes have letter prefixes. For example, codes starting with 'J' are related to diseases of the respiratory system.
# ICD-9 codes do not have letter prefixes, so there is no need to filter the years, except maybe for computational efficiency).
# Still, the following query runs fast with only 4GB of RAM and 2 cores on a FASSE compute node sustaining a remote desktop.

medpar_hospitalizations_prefix = '../data/input/mbsf_medpar_denom/medpar_hospitalizations'

conn = duckdb.connect()
icd_code_query = f"""
    WITH icd_diag AS (
    SELECT UNNEST(diagnoses) as diag
    FROM '{medpar_hospitalizations_prefix}_*.parquet'
    )

    SELECT DISTINCT diag 
    FROM icd_diag
    WHERE diag LIKE 'J%'
    """
conn.execute(icd_code_query).fetchdf()

In [7]:
conn.close()